<a href="https://colab.research.google.com/github/Yandarma/data-analytics-20101048/blob/main/spam_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import nltk
import spacy
import string
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/spam_ham_dataset.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      5171 non-null   object
 1   text       5171 non-null   object
 2   label_num  5171 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 121.3+ KB


1. All letters are converted from uppercase to lowercase
2. All characters and punctuation marks are deleted except
for the alphabets in Turkish and English.
3. Numbers are deleted.
4. Certain abbreviations in the text are expanded.
5. The words are split according to the white space.
6. Every word obtained as a result of splitting is separated
into its roots utilizing Zemberek [22], which is the
Turkish natural language processing library.
7. Stop words are deleted.

In [4]:
# All letters are converted from uppercase to lowercase
df["text"] = df["text"].str.lower()
df.head()

,label,text,label_num
0,ham,subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"subject: photoshop , windows , office . cheap ...",1
4,ham,subject: re : indian springs\r\nthis deal is t...,0


In [5]:
# remove subject: re :
df['text'].replace('subject:','',regex=True, inplace = True)
df['text'].replace('subject: re :','',regex=True, inplace = True)
df['text'].replace('re :','',regex=True, inplace = True)
df.head()

,label,text,label_num
0,ham,enron methanol ; meter # : 988291\r\nthis is ...,0
1,ham,"hpl nom for january 9 , 2001\r\n( see attache...",0
2,ham,"neon retreat\r\nho ho ho , we ' re around to ...",0
3,spam,"photoshop , windows , office . cheap . main t...",1
4,ham,indian springs\r\nthis deal is to book the t...,0


In [6]:
# punctuation marks are deleted
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
df["text"] = df["text"].apply(lambda text: remove_punctuation(text))
df.head()

,label,text,label_num
0,ham,enron methanol meter 988291\r\nthis is a f...,0
1,ham,hpl nom for january 9 2001\r\n see attached ...,0
2,ham,neon retreat\r\nho ho ho we re around to th...,0
3,spam,photoshop windows office cheap main trend...,1
4,ham,indian springs\r\nthis deal is to book the t...,0


In [7]:
# Numbers are deleted.
df['text'].replace('[0-9]','',regex=True, inplace = True)
df.head()

,label,text,label_num
0,ham,enron methanol meter \r\nthis is a follow ...,0
1,ham,hpl nom for january \r\n see attached file ...,0
2,ham,neon retreat\r\nho ho ho we re around to th...,0
3,spam,photoshop windows office cheap main trend...,1
4,ham,indian springs\r\nthis deal is to book the t...,0


In [8]:
# remove newline
df['text'].replace('\r','',regex=True, inplace = True)
df['text'].replace('\n','',regex=True, inplace = True)
df[100:105]

,label,text,label_num
100,ham,help brian who do i talk to about getting dar...,0
101,ham,fw ena origination track original messag...,0
102,ham,password resetthis is a generated email do n...,0
103,spam,lose powerful weightloss now available whe...,1
104,spam,smart inksystemwww compusoft gr,1


In [9]:
# remove multiple whitespace
df['text'] = df['text'].replace(r' +', ' ', regex=True)
df[100:105]

,label,text,label_num
100,ham,help brian who do i talk to about getting dar...,0
101,ham,fw ena origination track original message fro...,0
102,ham,password resetthis is a generated email do no...,0
103,spam,lose powerful weightloss now available where ...,1
104,spam,smart inksystemwww compusoft gr,1


In [10]:
# from sklearn.model_selection import train_test_split as tts
# from sklearn.feature_extraction.text import CountVectorizer

In [11]:
# vectorizer = CountVectorizer()
# X_train,X_test, y_train,y_test = tts(df['text'],df['label_num'], train_size=0.8, random_state=42)
# X_train = vectorizer.fit_transform(X_train)
# X_test = vectorizer.transform(X_test)

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Ditemukan %s token unik' %len(word_index))

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print("Ukuran tensor ", X.shape)

Ditemukan 65204 token unik
Ukuran tensor  (5171, 250)


In [13]:
Y = pd.get_dummies(df['label_num']).values
print('Shape label', Y.shape)

Shape label (5171, 2)


In [14]:
from sklearn.model_selection import train_test_split as tts

In [15]:
X_train,X_test, y_train,y_test = tts(X,Y, train_size=0.9, random_state=42)
X_train.shape,X_test.shape, y_train.shape,y_test.shape

((4653, 250), (518, 250), (4653, 2), (518, 2))

In [16]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.5))
model.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5

batch_size = 20

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
210/210 [==============================] - 120s 556ms/step - loss: 0.2743 - accuracy: 0.8694 - val_loss: 0.1470 - val_accuracy: 0.9356
Epoch 2/5
210/210 [==============================] - 109s 520ms/step - loss: 0.0526 - accuracy: 0.9811 - val_loss: 0.0620 - val_accuracy: 0.9785
Epoch 3/5
210/210 [==============================] - 110s 522ms/step - loss: 0.0176 - accuracy: 0.9947 - val_loss: 0.0486 - val_accuracy: 0.9807
Epoch 4/5
210/210 [==============================] - 108s 516ms/step - loss: 0.0063 - accuracy: 0.9979 - val_loss: 0.0636 - val_accuracy: 0.9807
Epoch 5/5
210/210 [==============================] - 108s 514ms/step - loss: 0.0018 - accuracy: 0.9995 - val_loss: 0.0659 - val_accuracy: 0.9807


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 5,080,602
Trainable params: 5,080,602
Non-trainable params: 0
_________________________________________________________________


In [18]:
X_train,X_test, y_train,y_test = tts(X,Y, train_size=0.8, random_state=42)
X_train.shape,X_test.shape, y_train.shape,y_test.shape

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.5))
model.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5

batch_size = 20

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
187/187 [==============================] - 101s 521ms/step - loss: 0.2725 - accuracy: 0.8716 - val_loss: 0.1044 - val_accuracy: 0.9686
Epoch 2/5
187/187 [==============================] - 97s 518ms/step - loss: 0.0668 - accuracy: 0.9790 - val_loss: 0.0924 - val_accuracy: 0.9710
Epoch 3/5
187/187 [==============================] - 97s 518ms/step - loss: 0.0292 - accuracy: 0.9933 - val_loss: 0.0671 - val_accuracy: 0.9734
Epoch 4/5
187/187 [==============================] - 96s 516ms/step - loss: 0.0089 - accuracy: 0.9979 - val_loss: 0.0595 - val_accuracy: 0.9831
Epoch 5/5
187/187 [==============================] - 97s 519ms/step - loss: 0.0031 - accuracy: 0.9997 - val_loss: 0.0644 - val_accuracy: 0.9831


In [19]:
X_train,X_test, y_train,y_test = tts(X,Y, train_size=0.7, random_state=42)
X_train.shape,X_test.shape, y_train.shape,y_test.shape

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.5))
model.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5

batch_size = 20

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
163/163 [==============================] - 89s 531ms/step - loss: 0.3289 - accuracy: 0.8407 - val_loss: 0.1293 - val_accuracy: 0.9448
Epoch 2/5
163/163 [==============================] - 84s 518ms/step - loss: 0.0603 - accuracy: 0.9782 - val_loss: 0.0886 - val_accuracy: 0.9669
Epoch 3/5
163/163 [==============================] - 84s 517ms/step - loss: 0.0188 - accuracy: 0.9948 - val_loss: 0.0725 - val_accuracy: 0.9807
Epoch 4/5
163/163 [==============================] - 85s 520ms/step - loss: 0.0101 - accuracy: 0.9972 - val_loss: 0.0975 - val_accuracy: 0.9696
Epoch 5/5
163/163 [==============================] - 85s 519ms/step - loss: 0.0050 - accuracy: 0.9988 - val_loss: 0.0790 - val_accuracy: 0.9862


In [20]:
X_train,X_test, y_train,y_test = tts(X,Y, train_size=0.5, random_state=42)
X_train.shape,X_test.shape, y_train.shape,y_test.shape

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.5))
model.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5

batch_size = 20

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
117/117 [==============================] - 64s 521ms/step - loss: 0.3982 - accuracy: 0.8130 - val_loss: 0.1753 - val_accuracy: 0.9305
Epoch 2/5
117/117 [==============================] - 60s 517ms/step - loss: 0.0754 - accuracy: 0.9746 - val_loss: 0.0982 - val_accuracy: 0.9575
Epoch 3/5
117/117 [==============================] - 61s 518ms/step - loss: 0.0229 - accuracy: 0.9936 - val_loss: 0.0936 - val_accuracy: 0.9691
Epoch 4/5
117/117 [==============================] - 61s 518ms/step - loss: 0.0086 - accuracy: 0.9983 - val_loss: 0.1155 - val_accuracy: 0.9575
Epoch 5/5
117/117 [==============================] - 61s 520ms/step - loss: 0.0060 - accuracy: 0.9987 - val_loss: 0.1268 - val_accuracy: 0.9537
